# Imports

In [1]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import requests
import urllib.request
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup

# Functions

In [2]:
def find_cards(button):
    table_output.clear_output()
    data_dict = {
        'Card': [], 
        'Hobbymaster': [], 
        'Baydragon': [], 
        'Spellbound': [], 
        'Magic at Willis': [], 
        'Goblin Games': [],
        'Iron Knight Gaming': []
    }
    cards = card_input.value.split('\n')
    total_value = []
    not_found = []
    for card in cards:
        results_output.clear_output()
        with results_output:
            display(widgets.HTML(f'Searching for <b>{card}<b>'))
        data_dict['Card'].append(card)
        hm_price = search_hm(card)   
        bd_price = search_bd(card)
        sb_price = search_sb(card)
        mw_price = search_mw(card)
        gg_price = search_gg(card)
        ik_price = search_ik(card)
        price_list = [price for price in (hm_price, bd_price, sb_price, mw_price, gg_price, ik_price) if price > 0]
        if len(price_list) > 0:
            lowest_price = min(price_list)
            total_value.append(lowest_price)
        else:
            lowest_price = 0
            not_found.append(card)        
        for shop, price in {
            'Hobbymaster': hm_price, 
            'Baydragon': bd_price, 
            'Spellbound': sb_price, 
            'Magic at Willis': mw_price,
            'Goblin Games': gg_price,
            'Iron Knight Gaming': ik_price
        }.items():
            data_dict[shop].append(price)
    global results_df
    results_df = pd.DataFrame(data_dict)
    results_output.clear_output()
    if len(not_found) > 0:
        cards_not_found = '| Could not find: ' + ', '.join(not_found)
    else:
        cards_not_found = ''
    with results_output:
        display(widgets.HTML(f"Total cost: <b>NZ${sum(total_value)}</b> {cards_not_found}"))
    with table_output:
        display(results_df.style.apply(highlight_min, axis=1, subset=['Hobbymaster', 'Baydragon', 'Spellbound', 'Magic at Willis', 'Goblin Games', 'Iron Knight Gaming']))
        
def highlight_min(data, color='green'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)



def search_hm(card_name):
    hm_api = 'https://hobbymaster.co.nz/cards/get-cards?foil=0&lang=&game=1&_search=true&sidx=set&sord=desc&name='
    url = hm_api + card_name
    response = requests.get(url)
    data = response.json()
    price_list = []
    if 'rows' in data:
        for item in data['rows']:
            if card_name.lower() in item['cell'][0].lower():
                if item['cell'][12] == 0:
                    continue
                else:
                    price = item['cell'][10]
                    price = float(price.replace('$', '').replace('!', ''))
                    price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)
    else:
        price = np.nan
    return price

def search_bd(card_name):
    bd_api = 'https://www.baydragon.co.nz/search/category/01?searchType=single&searchString='
    url = bd_api + card_name
    html_content = requests.get(url).text
#     soup = BeautifulSoup(html_content, "lxml")
    soup = BeautifulSoup(html_content)
    div = soup.find('div', attrs={'class': 'tcgSingles'})
    table = div.find('table')
    tds = table.find_all('td')
    n = 0
    price_list = []
    for td in tds:
        n += 1
        if n > len(tds):
            break
        if 'NZ$' in td.text:
            if card_name in tds[n-6].text:
                if int(tds[n].text) > 0:
                    price = float(td.text.replace('NZ$', ''))
                    price_list.append(price)
    if len(price_list) == 0:
        price = np.nan
    else:
        price = min(price_list)
    return price

def search_gg(card_name):
    html_content = requests.get(f'https://goblingames.nz/search?q={card_name}').text
    soup = BeautifulSoup(html_content)
    products = soup.find_all('div', attrs={'class': 'productCard__lower'})
    price_list = []
    for product in products:
        p_list = product.find_all('ul')
        for item in p_list:
            instock = item.find_all('li')
            if len(instock):
                for i in instock:
                    title = i.attrs['data-producttitle']
                    price = int(i.attrs['data-price']) / 100
                if card_name.lower() in title.lower():
                    price_list.append(price)
    if len(price_list) == 0:
        price = np.nan
    else:
        price = min(price_list)
    return price

def search_shopify(url, card_name):
    html_content = requests.get(url).text
#     soup = BeautifulSoup(html_content, "lxml")
    soup = BeautifulSoup(html_content)
    products = soup.find_all('div', attrs={'class': 'product Norm'})
    price_list = []
    for product in products:
        title = product.find('p', attrs='productTitle').text.replace('\n', ' ').strip()
        price = product.find('p', attrs='productPrice').text.replace('\n', ' ').strip()
        if card_name.lower() in title.lower():
            if price == 'Varies':
                prices = product.parent.find('div', attrs={'class': 'buyWrapper'}).find_all('p')
                for p in prices:
                    try:
                        price = float(p.text.split('$')[1])
                    except:
                        continue
                    price_list.append(price)
            elif '$' in price:
                try:
                    price = float(price.replace('$', ''))
                except:
                    continue
                price_list.append(price)
            else:
                if price != 'Sold Out':
                    price_list.append(price)
    if len(price_list) == 0:
        price = np.nan
    else:
        price = min(price_list)

    return price

def search_sb(card_name):
    price = search_shopify(f'https://spellboundgames.co.nz/search?q={card_name}', card_name)
    return price
    
def search_mw(card_name):
    price = search_shopify(f'https://magicatwillis.co.nz/search?q={card_name}', card_name)
    return price

def search_ik(card_name):
    price = search_shopify(f'https://ironknightgaming.co.nz/search?q={card_name}', card_name)
    return price

# User Interface

In [3]:
card_input = widgets.Textarea(placeholder='Enter a cards on new lines', rows=5)
go_button = widgets.Button(description='Find prices', button_style='')
results_output = widgets.Output()
table_output = widgets.Output()
display(card_input, go_button, results_output, table_output)
go_button.on_click(find_cards)

Textarea(value='', placeholder='Enter a cards on new lines', rows=5)

Button(description='Find prices', style=ButtonStyle())

Output()

Output()

In [5]:
card_input = widgets.Textarea(placeholder='Enter a cards on new lines', rows=5)
go_button = widgets.Button(description='Find prices', button_style='')
results_output = widgets.Output()
table_output = widgets.Output()
display(card_input, go_button, results_output, table_output)
go_button.on_click(find_cards)

Textarea(value='', placeholder='Enter a cards on new lines', rows=5)

Button(description='Find prices', style=ButtonStyle())

Output()

Output()

In [9]:
go_button.on_click(find_cards)


,Card,Hobbymaster,Baydragon,Spellbound,Magic at Willis,Goblin Games,Iron Knight Gaming
0,Abjure,NaN,NaN,NaN,NaN,NaN,0.50
1,Act of Treason,0.2,0.20,0.4,NaN,0.30,0.30
2,Arcane Signet,6.1,4.65,7.2,NaN,41.50,NaN
3,Ashnod's Altar,13.6,NaN,NaN,NaN,NaN,NaN
4,Bastion of Remembrance,NaN,NaN,NaN,NaN,NaN,NaN
5,Blood Funnel,1.4,NaN,1.5,1.1,5.50,NaN
6,Blood for Bones,NaN,NaN,NaN,NaN,NaN,NaN
7,Bloodshot Cyclops,0.7,NaN,NaN,NaN,NaN,NaN
8,Bloodsoaked Altar,0.5,0.38,0.8,NaN,0.50,NaN
9,Breath of Fury,NaN,NaN,NaN,NaN,3.00,NaN
